In [1]:
import os
import json
import openai
import asyncio
import numpy as np
import nest_asyncio
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, models

from src.openai_processor import chatgpt_response
from src.data_processor import translate_and_convert_to_string, process_vision_result, extract_workstyle_info

In [2]:
nest_asyncio.apply()
DATA_PATH = "./data/comments_sample.json"
GT_PATH = "./data/amy_gt.json"

PROMPT_VER = 2
N_ITER = 10 ## 50, 100
THRESHOLD = 0.7
EMBEDDING_MODEL_ID = "nlpai-lab/KoE5" ## "dragonkue/bge-m3-ko, "dragonkue/bge-m3-ko"

In [3]:
if PROMPT_VER == 1:
    from src.prompt_processor import generate_vision_prompt, generate_workstyle_prompt, generate_summary_prompt
elif PROMPT_VER == 2:
    from src.prompt_processor_v2 import generate_vision_prompt, generate_workstyle_prompt, generate_summary_prompt

In [4]:
with open(DATA_PATH, 'r', encoding='utf-8') as file:
    hr_data_dict = json.load(file)

with open(GT_PATH, 'r', encoding="utf-8") as file:
    gt_data_dict = json.load(file)

In [5]:
hr_data_dict

{'visionResult': {'company': {'keyWord': {'혁신': 5.0,
    '도전': 5.0,
    '사회공헌': 3.2,
    '즐거움': 3.5,
    '창조': 5.0,
    '전문성': 3.7}},
  'compute': {'keyWord': {'문제해결': 0.0,
    '상생': 2.5,
    '혁신': 3.64,
    '사회공헌': 1.67,
    '고객': 2.0,
    '즐거움': 2.22,
    '창조': 5.0,
    '성장': 3.08,
    '인재': 3.25,
    '소통': 2.31,
    '성과': 3.08,
    '최고지향': 3.13,
    '도전': 3.85,
    '신속성': 1.92,
    '열정': 2.5,
    '전문성': 1.36}}},
 'workstyleResult': {'company': {'keyWord': {'촉진자형': 5.0,
    '분석형': 5.0,
    '조력자형': 5.0,
    '논리형': 5.0,
    '협력형': 5.0,
    '겸손형': 5.0},
   'totalScore': 30.0},
  'compute': {'keyWord': {'촉진자형': 2.14,
    '분석형': 2.6,
    '조력자형': 1.88,
    '논리형': 2.22,
    '협력형': 3.08,
    '겸손형': 2.14},
   'totalScore': 14.06},
  'rate': 0.0},
 'summaryResult': {'additionalInformation': {'조기 퇴사 가능성': '보통',
   '입사 후 적응 기간': '짧음',
   '채용 권장 수준': '짧음'},
  'fued': ['과몰입형'],
  'recruitentQuestions': {'5) 사고방식이 기업 비전,가치관에 부합하는가?': '보통이다',
   '2) 타 팀, 타 구성원과의 원만한 협업을 기대할 수 있는가?': '아니다',
   '3) 경영

In [6]:
processed_data_summary = translate_and_convert_to_string(hr_data_dict['summaryResult'])
vision_data = process_vision_result(hr_data_dict['visionResult'], hr_data_dict['summaryResult'])
workstyle_data = extract_workstyle_info(hr_data_dict['workstyleResult'], hr_data_dict['summaryResult'])

In [7]:
vision_prompt = generate_vision_prompt(vision_data)
workstyle_prompt = generate_workstyle_prompt(workstyle_data)
summary_prompt = generate_summary_prompt(processed_data_summary)

In [8]:
async def generate_comments(prompts, num_samples=3):
    results = []
    for i in range(num_samples):
        vision_comment, workstyle_comment, summary_comment = await asyncio.gather(
            chatgpt_response(prompts["vision_prompt"]),
            chatgpt_response(prompts["workstyle_prompt"]),
            chatgpt_response(prompts["summary_prompt"]),
        )
        results.append({
            "vision_comment": vision_comment,
            "workstyle_comment": workstyle_comment,
            "summary_comment": summary_comment
        })
    return results

In [9]:
prompts = {
    "vision_prompt": generate_vision_prompt(vision_data),
    "workstyle_prompt": generate_workstyle_prompt(workstyle_data),
    "summary_prompt": generate_summary_prompt(processed_data_summary)
}

In [10]:
loop = asyncio.get_event_loop()
comments = loop.run_until_complete(generate_comments(prompts, num_samples=N_ITER))

In [11]:
df = pd.DataFrame(comments)

os.makedirs('./result', exist_ok=True)
df.to_csv(f"./result/comments_{N_ITER}.csv", index=False)

In [12]:
print(f"GT summary   : {gt_data_dict['summary_comment']}")
print(f"PRED summary : {df.iloc[0]['summary_comment']}")

GT summary   : 피검사자는 기업의 비전과 가치관에 높은 수준으로 정렬되어 있으며, 혁신성과 비전과의 높은 정렬을 가지고 있는 것으로 평가됩니다. 하지만, 작업 스타일과 기업의 평균적 기준에서 일부 부족한 상황이 상향 인재로 평가됩니다. 다만, 이문화성 영역에서 개선이 필요하다는 점이 도출되어, 팀 구성 및 조직 특성상 잠재적인 문제 발생 가능성을 미리 점검할 필요가 있습니다.
PRED summary : 피검사자는 입사 후 적응 기간이 짧아 빠르게 업무에 적응할 수 있을 것으로 보이나, 타 팀 및 구성원과의 협업에서 어려움을 겪을 가능성이 높습니다. 경영진 및 상급자와의 소통은 보통 수준이며, 기업의 일하는 방식과는 부합하지 않는 경향이 있습니다. 구성원들과의 소통은 원활하나, 소통 관계에서 이직 스트레스 요인이 존재합니다. 과몰입형 위험 성향이 감지되어 우려되며, 갈등 유발 요인을 확인하라. 종합적으로, 조기 퇴사 가능성은 보통 수준입니다.


In [13]:
print(f"GT vision   : {gt_data_dict['vision_comment']}")
print(f"PRED vision : {df.iloc[0]['vision_comment']}")

GT vision   : 피검사자는 기업이 중점적으로 추구하는 비전 요소 중 사회공헌, 창조, 고객 중심, 그리고 혁신에서 기업의 기대치와 높은 수준으로 정렬되어 있습니다. 또한 열정, 성장, 소통, 성실, 존중 등 다른 영역에서 기업의 기대치를 충족하는 높은 인재상을 보이고 있습니다. 다만, '최고지향' 및 '인식' 와 관련된 점수는 상대적으로 크게 나타납니다. 전반적으로 기업과의 가치관 정렬이 우수하며 평가됩니다. 결론적으로, 피검사자는 기업 비전과 높은 적합성을 보유하고 있습니다.
PRED vision : 피검사자는 기업의 핵심 비전 키워드인 '창조'에서 기업과 완벽하게 일치하여 긍정적인 평가를 받을 수 있습니다. 그러나 '도전'과 '혁신'에서는 기업의 기대치에 비해 다소 낮은 점수를 보이고 있어, 이 부분에서의 추가적인 발전이 필요할 것으로 보입니다. 특히 '문제해결'에서는 기업의 기대치와 큰 차이를 보이고 있어 우려가 됩니다. 반면, 피검사자는 '사회공헌'에서 기업의 기대치를 충족하지 못하지만, '창조'에서의 높은 점수는 긍정적입니다. 결론적으로, 피검사자와 기업 간 비전 fit은 검토가 필요합니다.


In [14]:
print(f"GT workstyle   : {gt_data_dict['workstyle_comment']}")
print(f"PRED workstyle : {df.iloc[0]['workstyle_comment']}")

GT workstyle   : 피검사자는 혁신성, 친화성, 유니크성, 스피드형 부문에서 피검사자는 기업의 비전과 높은 수준으로 정렬된 모습을 보입니다. 또한 성과, 도전, 관계성 영역에서는 평균 수준에 해당되는 비전을 보여주고 있습니다. 반면, 논리성, 신뢰형, 그리고 이문화성의 경우 기업의 비전과의 일부 차이를 보이는 것으로 보입니다. 전반적으로, 피검사자는 기업의 작업 스타일 상황 fit 은 평균적인 수준으로 평가됩니다.
PRED workstyle : 피검사자는 기업의 촉진자형, 분석형, 조력자형, 논리형, 협력형, 겸손형 업무 스타일과 비교했을 때 전반적으로 낮은 수준을 보이고 있습니다. 특히 조력자형과 논리형에서 기업의 기대치와 차이가 있으며, 협력형에서는 상대적으로 조금 더 높은 점수를 보이지만 여전히 기업의 기대에 미치지 못하고 있습니다. 이러한 차이는 피검사자가 기업의 업무 스타일에 적응하는 데 있어 추가적인 검토가 필요함을 시사합니다. 피검사자와 기업 간 업무 성향 fit은 검토 필요입니다.
